In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/mnt/home/surkov/conda_envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def print_hook(m,i,o):
    print(o)
    return o

In [51]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").cuda()
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

input_text = "translate into french: Tell me about the greatest scientist in the world"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

base_model.get_submodule("encoder.block.5.layer.0.SelfAttention.q").register_forward_hook(print_hook)

outputs = base_model.generate(input_ids, max_new_tokens=30)
print(tokenizer.decode(outputs[0]))

tensor([[[ 0.0653, -0.2262,  0.0242,  ..., -0.0565,  0.0307,  0.2187],
         [ 0.0558,  0.0318, -0.0228,  ..., -0.2404, -0.0032,  0.0370],
         [-0.0338, -0.0301, -0.0295,  ...,  0.1639, -0.0569,  0.0554],
         ...,
         [ 0.0487,  0.0923, -0.0801,  ..., -0.1878, -0.0219, -0.1405],
         [ 0.0523,  0.0016,  0.0901,  ...,  0.1566,  0.0726,  0.0215],
         [-0.2051,  0.0834,  0.1736,  ...,  0.0523,  0.0012,  0.3493]]],
       device='cuda:0')
<pad> Même s'il y a d'autres histoires d'histoires d'histoires d'


In [52]:
lora_config = dict(target_modules=[".q", ".v"], alpha=32)

In [53]:
from multilora import LoRAModel
model = LoRAModel(base_model, lora_config).cuda()

encoder.block.0.layer.0.SelfAttention.q
encoder.block.0.layer.0.SelfAttention.v
encoder.block.1.layer.0.SelfAttention.q
encoder.block.1.layer.0.SelfAttention.v
encoder.block.2.layer.0.SelfAttention.q
encoder.block.2.layer.0.SelfAttention.v
encoder.block.3.layer.0.SelfAttention.q
encoder.block.3.layer.0.SelfAttention.v
encoder.block.4.layer.0.SelfAttention.q
encoder.block.4.layer.0.SelfAttention.v
encoder.block.5.layer.0.SelfAttention.q
encoder.block.5.layer.0.SelfAttention.v
encoder.block.6.layer.0.SelfAttention.q
encoder.block.6.layer.0.SelfAttention.v
encoder.block.7.layer.0.SelfAttention.q
encoder.block.7.layer.0.SelfAttention.v
encoder.block.8.layer.0.SelfAttention.q
encoder.block.8.layer.0.SelfAttention.v
encoder.block.9.layer.0.SelfAttention.q
encoder.block.9.layer.0.SelfAttention.v
encoder.block.10.layer.0.SelfAttention.q
encoder.block.10.layer.0.SelfAttention.v
encoder.block.11.layer.0.SelfAttention.q
encoder.block.11.layer.0.SelfAttention.v
encoder.block.12.layer.0.SelfAttenti

In [54]:
model.adapter_ids.data = torch.tensor([0]).to('cuda')
outputs = model.base_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

tensor([[[ 0.0653, -0.2262,  0.0242,  ..., -0.0565,  0.0307,  0.2187],
         [ 0.0558,  0.0318, -0.0228,  ..., -0.2404, -0.0032,  0.0370],
         [-0.0338, -0.0301, -0.0295,  ...,  0.1639, -0.0569,  0.0554],
         ...,
         [ 0.0487,  0.0923, -0.0801,  ..., -0.1878, -0.0219, -0.1405],
         [ 0.0523,  0.0016,  0.0901,  ...,  0.1566,  0.0726,  0.0215],
         [-0.2051,  0.0834,  0.1736,  ...,  0.0523,  0.0012,  0.3493]]],
       device='cuda:0')
<pad> Même s'il y a d'autres histoires d'


In [55]:
file = torch.load('adapter_model.bin', weights_only=False)

In [56]:
file.keys()

dict_keys(['base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_B.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_A.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_B.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_B.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.v.lora_A.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.v.lora_B.weight', 'base_model.model.encoder.block.3.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.encoder.b

In [57]:
for key, value in file.items():
    if key.endswith('lora_A.weight'):
        aorb = 'A'
        stem = key.strip('.lora_A.weight')
    else:
        aorb = 'B'
        stem = key.strip('.lora_B.weight')
    path = stem + '.module2.' + aorb
    path = path.replace('.model.', '.')
    print(key)
    print(value.shape)
    model.get_parameter(path).data = value.T.unsqueeze(0)

base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.weight
torch.Size([16, 1024])
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.weight
torch.Size([1024, 16])
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.weight
torch.Size([16, 1024])
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.weight
torch.Size([1024, 16])
base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_A.weight
torch.Size([16, 1024])
base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_B.weight
torch.Size([1024, 16])
base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_A.weight
torch.Size([16, 1024])
base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_B.weight
torch.Size([1024, 16])
base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_A.weight
torch.Size([16, 1024])
base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_B.weight
torch.Size([1024, 16])
base_model.model.encoder.block.2.layer.0.SelfAttention.v.lor

In [58]:
model.adapter_ids.data = torch.tensor([0]).to('cuda')
outputs = model.base_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

tensor([[[ 6.6186e-02, -2.2885e-01,  2.4074e-02,  ..., -5.1724e-02,
           2.8821e-02,  2.1549e-01],
         [ 5.3973e-02,  3.7252e-02, -2.8463e-02,  ..., -2.3661e-01,
          -4.8198e-03,  2.3165e-02],
         [-3.3200e-02, -3.1014e-02, -3.0228e-02,  ...,  1.6477e-01,
          -5.7422e-02,  5.5504e-02],
         ...,
         [ 4.8742e-02,  9.2203e-02, -8.0042e-02,  ..., -1.8773e-01,
          -2.1930e-02, -1.4047e-01],
         [ 5.1625e-02, -1.1196e-04,  8.8337e-02,  ...,  1.5608e-01,
           7.2920e-02,  2.4301e-02],
         [-2.0471e-01,  8.1783e-02,  1.7320e-01,  ...,  5.3410e-02,
          -7.8545e-06,  3.4848e-01]]], device='cuda:0')
<pad> Même en Allemagne, il n'y a pas de l'
